In [98]:
from neo4j import GraphDatabase

In [99]:
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "qwer123456")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver

In [100]:
driver.execute_query("match (n) detach delete n", database_='neo4j')  # 删除所有节点和关系

driver.execute_query("create"
                     "(:moive{name:'无间道', type:'犯罪', score:8.7}),"
                     "(:moive{name:'大内密探零零发', type:'搞笑', score:9.4}),"
                     "(:moive{name:'我和我的祖国', type:'主旋律', score:7.7})", database_="neo4j")
driver.execute_query("create (:person{name:'dmm', age:4})-[:love]->(:moive{name:'西游记'})", database_='neo4j')
driver.execute_query("create (:person{name:'xiaomai', age:4})-[:love{level:'must', time:5}]->(:moive{name:'西游记'})", database_='neo4j')
driver.execute_query("create (:person{name:'dmm', age:4})-[:dislike{level:'must', time:4}]->(:moive{name:'三国演义'})", database_='neo4j')
driver.execute_query("create (:dog{name:'wangcai', age:1})-[:love]->(:find{name:'狗粮'})", database_='neo4j')

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x000001E4E50EEC20>, keys=[])

In [101]:
# 指定关系类型为love
# 返回开始节点(n),结尾节点(m),开始与结尾节点的关系(r)
r0 = driver.execute_query("match (n)-[r:love]->(m) return n, r, m", database_='neo4j').records
print(r0)
for i in r0:
    print(i.data())  # 字典

    values = i.values()
    print(values)
    print(type(values[0]))
    print(values[0].labels)
    print(values[0].items())

    print(type(values[1]))
    print(values[1].type)
    print(values[1].items())

[<Record n=<Node element_id='15' labels=frozenset({'person'}) properties={'name': 'dmm', 'age': 4}> r=<Relationship element_id='14' nodes=(<Node element_id='15' labels=frozenset({'person'}) properties={'name': 'dmm', 'age': 4}>, <Node element_id='22' labels=frozenset({'moive'}) properties={'name': '西游记'}>) type='love' properties={}> m=<Node element_id='22' labels=frozenset({'moive'}) properties={'name': '西游记'}>>, <Record n=<Node element_id='23' labels=frozenset({'person'}) properties={'name': 'xiaomai', 'age': 4}> r=<Relationship element_id='15' nodes=(<Node element_id='23' labels=frozenset({'person'}) properties={'name': 'xiaomai', 'age': 4}>, <Node element_id='24' labels=frozenset({'moive'}) properties={'name': '西游记'}>) type='love' properties={'level': 'must', 'time': 5}> m=<Node element_id='24' labels=frozenset({'moive'}) properties={'name': '西游记'}>>, <Record n=<Node element_id='16' labels=frozenset({'dog'}) properties={'name': 'wangcai', 'age': 1}> r=<Relationship element_id='17' n

In [102]:
# 指定关系time属性为4
# 只返回开始与结尾节点的关系(r)
r0 = driver.execute_query("match ()-[r{time:4}]->() return r", database_='neo4j').records
for i in r0:
    print(i.data())

{'r': ({}, 'dislike', {})}


In [103]:
# 关系为任意;开始节点类型为person,结尾节点为任意
# 只返回开始节点(m)
r0 = driver.execute_query("match (m:person)-[r]->() return m", database_='neo4j').records
for i in r0:
    print(i.data())

{'m': {'name': 'dmm', 'age': 4}}
{'m': {'name': 'dmm', 'age': 4}}
{'m': {'name': 'xiaomai', 'age': 4}}


In [104]:
# 关系为任意;开始节点类型name属性为xiaomai,结尾节点为任意
# 返回查询到的所有节点、关系
r0 = driver.execute_query("match (a:person{name:'xiaomai'})-[r]->(b) return *", database_='neo4j').records
for i in r0:
    print(i.data())

{'a': {'name': 'xiaomai', 'age': 4}, 'b': {'name': '西游记'}, 'r': ({'name': 'xiaomai', 'age': 4}, 'love', {'name': '西游记'})}


In [105]:
# 指定关系类型为love;开始节点类型name属性为xiaomai,结尾节点类型为moive
r0 = driver.execute_query("match (a:person{name:'xiaomai'})-[r:love]->(b:moive) return a, b", database_='neo4j').records
for i in r0:
    print(i.data())

{'a': {'name': 'xiaomai', 'age': 4}, 'b': {'name': '西游记'}}


In [106]:
# 关系为任意;开始节点,结尾节点为任意
r0 = driver.execute_query("match (m)-[r]->(n) return m, n, r", database_='neo4j').records
for i in r0:
    print(i.data())

{'m': {'name': 'dmm', 'age': 4}, 'n': {'name': '三国演义'}, 'r': ({'name': 'dmm', 'age': 4}, 'dislike', {'name': '三国演义'})}
{'m': {'name': 'dmm', 'age': 4}, 'n': {'name': '西游记'}, 'r': ({'name': 'dmm', 'age': 4}, 'love', {'name': '西游记'})}
{'m': {'name': 'wangcai', 'age': 1}, 'n': {'name': '狗粮'}, 'r': ({'name': 'wangcai', 'age': 1}, 'love', {'name': '狗粮'})}
{'m': {'name': 'xiaomai', 'age': 4}, 'n': {'name': '西游记'}, 'r': ({'name': 'xiaomai', 'age': 4}, 'love', {'name': '西游记'})}
